In [45]:
import sqlite3
import pandas as pd
from datetime import date 
from datetime import datetime
import numpy as np 

In [46]:
#Conexão com o banco 
banco = sqlite3.connect("desafio-tecnico.db")
cur = banco.cursor()

In [47]:
# 1. Dados das Contas  

query = cur.execute('''select 
	a.id, 
	a.account_number, 
	a.birth, 
	a.occupation, 
	a.email, 
	c.state,	
	c.city,
	b.level
from accounts a 
inner join address c on a.address_id = c.id 
left join levels b on a.account_number = b.account_number''').fetchall()


In [48]:
df_acc = pd.DataFrame(query, columns = ['id', 'account_number', 'birth', 'occupation', 'email', 'state', 'city', 'level'])


In [49]:
# 2. Dados dos Boletos   

query = cur.execute('''select 
                        account_number,
                        status,
                        count(*) as count_charges, 
                        (sum(value)/100) as total_charges
                    from charges 
                    group by account_number, status
                    order by account_number''').fetchall()

df_charge = pd.DataFrame(query, columns = ['account_number', 'status', 'count_charges', 'total_charges'])


agp = df_charge.groupby(['account_number']).sum(numeric_only = True)

tbl_charge_status = pd.merge(df_charge, agp, on='account_number')

tbl_charge_status['Frequencia %'] =  round(tbl_charge_status['count_charges_x']*100/ tbl_charge_status['count_charges_y'], 2)


In [50]:
tbl_charge_paid = tbl_charge_status.loc[tbl_charge_status['status'] == 'paid'].copy()

tbl_charge_paid = tbl_charge_paid[['account_number', 'count_charges_x', 'total_charges_x', 'Frequencia %']]

tbl_charge_paid.rename(columns = {'count_charges_x': 'count_charges_paid', 'total_charges_x': 'total_charges_paid'\
                                         , 'Frequencia %': 'Perc_charges_paid'}, inplace = True)


In [51]:

# filtrando boletos não pagos 

tbl_charge_unpaid = tbl_charge_status.loc[tbl_charge_status['status'] == 'unpaid'].copy()

tbl_charge_unpaid = tbl_charge_unpaid[['account_number', 'count_charges_x', 'total_charges_x', 'Frequencia %']]

tbl_charge_unpaid.rename(columns = {'count_charges_x': 'count_charges_unpaid', 'total_charges_x': 'total_charges_unpaid',\
                                'Frequencia %': 'Perc_charges_unpaid'}, inplace = True)


In [52]:
## unificando dados 

df_unif_charge = pd.merge(tbl_charge_unpaid, tbl_charge_paid, on='account_number', how='outer')
    

In [53]:
#### 3. Dados das transacoes

query = cur.execute('''select 
                    	a.account_number, 
                    	b.description,
                    	count(a.id) as count_trans,
                    	(sum(a.value)/100) as total_trans	
                    from transactions a 
                    inner join transaction_type b on a.transaction_type_id = b.id
                    group by a.account_number, b.description
                    order by account_number''').fetchall()
banco.close()
    
df_trans = pd.DataFrame(query, columns = ['account_number', 'description', 'count_trans', 'total_trans'])
  
agp = df_trans.groupby(['account_number']).sum(numeric_only = True)

tbl_trans = pd.merge(df_trans, agp, on='account_number')

tbl_trans['Frequencia %'] =  round(tbl_trans['count_trans_x']*100/ tbl_trans['count_trans_y'], 2)



In [54]:
# filtrando as transacoes br - boleto_recebido
 
tbl_trans_br = tbl_trans.loc[tbl_trans['description'] == 'boleto_recebido'].copy()

tbl_trans_br = tbl_trans_br[['account_number', 'total_trans_x', 'Frequencia %']]

tbl_trans_br.rename(columns = {'total_trans_x': 'total_trans_blt_rec'\
                                         , 'Frequencia %': 'Perc_blt_rec'}, inplace = True)



In [55]:

# filtrando as transacoes pxe - pix_enviado
 
tbl_trans_pxe = tbl_trans.loc[tbl_trans['description'] == 'pix_enviado'].copy()

tbl_trans_pxe = tbl_trans_pxe[['account_number', 'total_trans_x', 'Frequencia %']]

tbl_trans_pxe.rename(columns = {'total_trans_x': 'total_trans_pix_env'\
                                         , 'Frequencia %': 'Perc_pix_env'}, inplace = True)

 

In [56]:
 
# filtrando as transacoes pxr - pix_recebido
 
tbl_trans_pxr = tbl_trans.loc[tbl_trans['description'] == 'pix_recebido'].copy()

tbl_trans_pxr = tbl_trans_pxr[['account_number', 'total_trans_x', 'Frequencia %']]

tbl_trans_pxr.rename(columns = {'total_trans_x': 'total_trans_pix_rec'\
                                         , 'Frequencia %': 'Perc_pix_rec'}, inplace = True)

    

In [57]:
    
## unificando dados 

df_unif_trans = pd.merge(tbl_trans_br, tbl_trans_pxe, on='account_number', how='outer')
df_unif_trans = pd.merge(df_unif_trans, tbl_trans_pxr, on='account_number', how='outer')    

In [58]:
# 4. Base para realizar as previsões 

df = df_acc.copy()

df = df[['account_number', 'level']]

df = pd.merge(df, df_unif_charge, on='account_number', how='left')
df = pd.merge(df, df_unif_trans, on='account_number', how='left')

df['total_trans'] = df['total_trans_blt_rec'] + df['total_trans_pix_rec'] + df['total_trans_pix_env']

del df_acc , df_unif_charge, df_unif_trans

In [59]:
################# MODELO ###############

base = df.copy()
base.dropna(inplace=True)
#base.dropna(subset=['level'], inplace=True)


prev = df.copy()     
prev['level'] = df['level'].replace(np.nan, 0)
prev = prev.loc[prev['level'] == 0]
prev = prev.drop('level', axis = 1)

prev_total = prev.copy()

prev.dropna(inplace=True)

treino = base.copy()

In [60]:
#algoritmo de previsao 

y = treino['level']
x = treino.drop('level', axis = 1)

from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.4)

from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier()
modelo.fit(x_treino, y_treino)

resultado = modelo.score(x_teste, y_teste)
print("Acurácia:", resultado)


Acurácia: 0.5544871794871795


In [61]:
# realizando previsao 

prev['level'] = modelo.predict(prev)
prev = prev[['account_number', 'level']]


In [62]:
############### Avaliando os casos restantes ############

prev_total = prev_total[['account_number']]

prev_total = pd.merge(prev_total, prev, on='account_number', how='left')
prev_total['level'] = prev_total['level'].replace(np.nan, 0)
prev_total = prev_total.loc[prev_total['level'] == 0]

prev_total = prev_total.drop('level', axis = 1)

df_prev_definir = prev_total.copy()

df_prev_definir = pd.merge(df_prev_definir, df, on='account_number', how='left')


In [63]:
# Casos em que o unpaid é maior que o paid é F

df_prev_definir.loc[(df_prev_definir['Perc_charges_unpaid'] >= 50) \
                      | (df_prev_definir['Perc_charges_unpaid'] == '100'), 'level'] = 'F'   

df_prev_definir = df_prev_definir[['account_number', 'level']]

In [64]:
# Casos restantes D e Casos em que não há dados D 

df_prev_definir['level'] = df_prev_definir['level'].replace(np.nan, 'D')

In [65]:
# Resultado final 
                
previsao_final = pd.concat([prev, df_prev_definir])

In [66]:
## Classificação das contas 

previsao_final['fraud'] = np.nan

previsao_final.loc[(previsao_final['level']=='F'),'fraud'] = 1

previsao_final['fraud'] = previsao_final['fraud'].replace(np.nan, 0)
        
previsao_final.to_csv('previsao_final.csv', index=False)